> **Diagnosing Diabetic Retinopathy from Fundus Photos **

Catherine Ko; Computational Modeling and Simulation; March 6, 2020

**About Diabetic Retinopathy**

Diabetic Retinopathy is a diabetic eye disease that can lead to vision loss and blindness in extreme cases. It occurs when high blood sugar damages blood vessels in the retina (back of the eye). Diabetic retinopathy is typically diagnosed by ophthalmologists via fundus photography. They take a picture of your retina and look for the telltale signs as shown below:

![](https://www.researchgate.net/profile/Asiri_Wijesinghe/publication/303481072/figure/fig1/AS:394097530556416@1470971581841/Retinal-lesions-in-DR-such-as-microaneurysms-exudates-and-hemorrhages-regions-13.png)

Microaneyrsm - a tiny swelling in the side of a blood vessel characterized by small red dots on the retina

Hemmorages - bleeding from a ruptured blood vessel

Soft exudates (cotton-wool spots) - internal superficial leakage from retinal arteries

Hard exudates - extracellular lipid leaked from retinal capillaries characterized by yellow grains




From a Kaggle competition, I was able to access ~3,600 retina images. 

Each image in the training dataset was labeled with one of the following diagnoses: 

0 - No DR

1 - Mild

2 - Moderate

3 - Severe

4 - Proliferative DR

My goal for this project was to diagnosis whether a patient has diabetic retinopathy (DR) (and to what degree) from running these fundus photos through my machine learning algorithm.

First, I imported the data from the Kaggle competition by reading in the training csv.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from keras.preprocessing import image
import cv2
from tqdm import tqdm 
from PIL import Image, ImageEnhance

In [ ]:
N=3662 # number of images 

df_train = pd.read_csv("../input/aptos2019-blindness-detection/train.csv")

Let's take a look at a portion of the training data csv:

In [ ]:
df_train.head()

As shown above, the csv gives the image's id_code in the first column and the corresponding diagnosis for severity of diabetic retinopathy in the second column.

Now let's take a look at the distribution of diabetic retinopathy diagnosis by plotting a histogram of the counts and printing the raw counts:

In [ ]:
(counts,bin_edges,_)=plt.hist(df_train['diagnosis'],align='left',bins=5,edgecolor='purple',linewidth=1.5,color='lavender')
plt.title('Histogram of diabetic retinopathy diagnosis', fontsize=14,weight='bold')
plt.xticks(bin_edges[:-1], np.arange(0,5,1))

ax=plt.gca()
ax.set_xlabel('Diabetic Retinopathy Diagnosis',fontsize=12);
ax.set_ylabel('Frequency',fontsize=12);

df_train['diagnosis'].value_counts()

Let's take a look at the actual retina images we got from the Kaggle competition:

In [ ]:
paths='../input/aptos2019-blindness-detection/train_images/'+df_train['id_code'][:5]+'.png'
plt.figure(figsize=(20,4))
for index, (path,label) in enumerate(zip(paths,df_train['diagnosis'][:5])):
    plt.subplot(1,5,index+1)
    plt.imshow(Image.open(path))
    plt.title('Diagnosis: %i\n' % label, fontsize = 20)

**Image Preprocssing **

Before running the images through my machine learning algorithm, I wanted to standardize all the images and improve lighting conditions to make the telltale signs of diabetic retinopathy that we discussed earlier more obvious. 

Since these images come from a variety of clinics and imaging conditions, the sizes of the image are different and so I created a function called preprocess_image to resize the images to 224x224 and increase the image contrast by 1.5x. I chose to increase the contrast to make the telltale signs of diabetic retinopathy more obvious. 

In [ ]:
def preprocess_image(path, desired_size=224):
    '''
    resize image to desired size x desired size
    and also increase contrast by 1.5x
    '''
    im = Image.open(path)
    im = im.resize((desired_size, )*2,resample=Image.LANCZOS)
    
    # increase contrast of the images
    enhancer = ImageEnhance.Brightness(im)
    factor = 1.5 #factor > 1 increases the contrast
    im_output = enhancer.enhance(factor)
    return im_output

After creating the function in charge of preprocessing the images, I ran all 3,662 images through my preprocessing function and stored the image data in a 4D array called pics_data. 

In [ ]:
%%time 

# reading in images into the array pics_data
# https://www.kaggle.com/xhlulu/aptos-2019-densenet-keras-starter

# create an empty 4d array to store the images
pics_data = np.empty((N, 224, 224,3), dtype=np.uint8)

for i, image_name in enumerate(tqdm(df_train['id_code'][:N])):
    pics_data[i, :, :,:] = preprocess_image(f'../input/aptos2019-blindness-detection/train_images/{image_name}.png')


After processing my images, here's a glimpse of the images after preprocessing. Notice how the contrast is increased and the images are all the same size now. 

In [ ]:
plt.figure(figsize=(20,4))
for index, (path,label) in enumerate(zip(paths,df_train['diagnosis'][:5])):
    plt.subplot(1,5,index+1)
    plt.imshow(preprocess_image(path))
    plt.title('Diagnosis: %i\n' % label, fontsize = 20)

Here I'm just reformatting the shape of my arrays to make sure I can run it through the next steps.

In [ ]:
# store corresponding diagnoses in an array named y_data
y_data=np.array(df_train['diagnosis'][:N])

# print the shape of y_data array
y_data.shape

In [ ]:
# print the shape of pics_data
pics_data.shape

In [ ]:
# reshape y_data from 1D --> 2D 
y_data2D=y_data.reshape(N,1)

# reshape pics_data from 1D --> 2D 
pics_data2D=pics_data.reshape(N,150528)

In the next part, I split up the training images into a training set of images and a testing set of images. I also split up the corresponding diagnoses into a training set and testing set. We will use the training sets to train the model and use the testing sets to test the model and see how the model performs on data it hasn't seen before after training. 

In [ ]:
# split dataset into training and test sets in a way that is blind to the programmer
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(pics_data2D, y_data2D, test_size=0.25, random_state=0)

Finally, it is time to make the machine learning model! Below, I created a logistic regressor and trained it using my training datasets.

In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
# create logistic regressor
logisticRegr = LogisticRegression(random_state=0)
# train logistic regressor using training sets
logisticRegr.fit(x_train, y_train.ravel()) 

Now the logistic regressor has been trained! 
Time to use the newly-trained model to make diagnoses predictions on our testing set of images.

In [ ]:
predictions = logisticRegr.predict(x_test) # predict entire test set
y_test_reshaped=y_test.reshape(916,)
df = pd.DataFrame({'Actual':y_test_reshaped,'Predicted':predictions})
df.tail()

Let's take a look at how accurate our model was. Below I printed some stats that will help us evaluate the error of this model.

In [ ]:
# evaluating the algorithm

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

Accuracy of the logistic regression model above is ~72% as shown in the number in the f1-score column and accuracy row. The f1-score for diagnosis 0 (no DR) was the highest at 93%, then diagnosis of 2 (moderate DR) at 61%, diagnosis of 1 (mild DR) at 45%, and diagnosis of 4 (proliferative DR) at 23%, and finally 3 (severe DR) at 14%. So in general, the model was better at correctly predicting you if did not have DR. Since the model's f1-score was not that high for people with severe and proliferative DR, the model would be likely to predict that you have less severe DR than you might actually have (more false negatives in a sense).

This would be less ideal than having a model with more false positives because this means that people with severe/proliferative DR are more likely to not be concerned enough with the actual state of their eye health.

**How exactly is the model being trained?**

In logistic regression (a form of supervised learning), when we are training the dataset, the program takes a whole bunch of x vectors from x_train set and runs them through the model $\theta$. Then the program compares the calculated hypothesis (predicted) values h(x) and compares them to the actual y values stored in y_train. Mathematically, the computer calculates the cost function which is the sum of all the squared differences between actual - predicted y. 

Then the algorithm uses gradient descent and partial derivative calculus stuff behind the scenes to change the numbers in the model vector $\theta$ until the cost function (error) is minimized! 

I'm using logistic regression as opposed to linear regression (another form of supervised learning) since I want the output to be categorical (either 0,1,2,3,4 depending on the severity of DR). 


**Future Directions**

Some future directions for this project could be to improve the accuracy of my logistic regression and possibly create a neural network. I tried some other logistic regressors (with different number of training cycles and solver methods) and neural nets below but didn't have the time to bring those models to fruition.

** Some other logistic regressors **

In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression(random_state=0,max_iter=1000)
logisticRegr.fit(x_train, y_train.ravel()) # train regressor 
# evaluating the algorithm
predictions = logisticRegr.predict(x_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression(random_state=0,solver='saga')
logisticRegr.fit(x_train, y_train.ravel()) # train regressor 
# evaluating the algorithm
predictions = logisticRegr.predict(x_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

** Trying out Neural Nets **

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(10,10,10),max_iter=20,random_state=0) # creates neural network
# hidden_layer_sizes creates 3 layers of 10 nodes each; just try different combos and see what is best
#max_iter = number of iterations of epochs (cycles of feed-forward and back propagation)
mlp.fit(x_train, y_train.ravel())
# make predictions to our test data
predictions=mlp.predict(x_test)
# evaluating the neural net algorithm

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(10,10,10),random_state=0) # creates neural network
# hidden_layer_sizes creates 3 layers of 10 nodes each; just try different combos and see what is best
#max_iter = number of iterations of epochs (cycles of feed-forward and back propagation)
mlp.fit(x_train, y_train.ravel())
# make predictions to our test data
predictions=mlp.predict(x_test)
# evaluating the neural net algorithm

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))